## Lanzamiento de instancias spot EC2

En este taller aprenderá los fundamentos de las instancias de spot de EC2 y las herramientas recomendadas para lanzar instancias de spot, comprobar la resistencia y ver el historial de precios.

In [1]:
import boto3
import json
import time

ec2 = boto3.client('ec2')
ec2_resource = boto3.resource('ec2')

region = ec2.meta.region_name
print("Current AWS region:", region)

Current AWS region: us-east-1


In [2]:
# Looks up the latest Amazon Linux 2 AMI
filters = [
    {
        'Name': 'name',
        'Values': ['amzn2-ami-kernel*gp2']
    },
    {
        'Name': 'virtualization-type',
        'Values': ['hvm']
    },
    {
        'Name': 'root-device-type',
        'Values': ['ebs']
    }
]

# Describe the images with the specified filters
response = ec2.describe_images(Owners=['amazon'], Filters=filters)
# Sort the images by CreationDate and get the latest ImageId
sorted_images = sorted(response['Images'], key=lambda x: x['CreationDate'], reverse=True)
latest_ami_id = sorted_images[0]['ImageId']

print(latest_ami_id)

ami-01eccbf80522b562b


In [3]:
# Specify the JSON file path
file_path = "launch-template-data.json"

# Read the JSON data from the file
with open(file_path, 'r') as file:
    data = json.load(file)

# Perform the text replacements
data["ImageId"] = latest_ami_id

# Write the modified JSON data back to the file
with open(file_path, 'w') as file:
    json.dump(data, file, indent=2)

#### Crear la plantilla de lanzamiento

In [4]:
# Specify the launch template name
template_name = "TemplateForWebServer"

# Read the launch template data from the JSON file
with open(file_path, 'r') as file:
    launch_template_data = json.load(file)
    
# Create the launch template
response = ec2.create_launch_template(
    LaunchTemplateName=template_name,
    VersionDescription="Initial version",
    LaunchTemplateData=launch_template_data
)

launch_template_id = response['LaunchTemplate']['LaunchTemplateId']
print("Launch template created with ID:", launch_template_id)

Launch template created with ID: lt-06aff29dace51fc9c


### Lanzamiento de instancias spot EC2 a través del grupo EC2 Auto Scaling

A la hora de adoptar instancias spot de EC2, le recomendamos que tenga en cuenta el grupo Amazon EC2 Auto Scaling (ASG), ya que ofrece las características de EC2 más actualizadas, como la selección del tipo de instancia basada en atributos, el reequilibrio de la capacidad, las políticas de escalado y muchas más funcionalidades.

Los grupos de Amazon EC2 Auto Scaling contienen una colección de instancias de Amazon EC2 que se tratan como una agrupación lógica a efectos de escalado y administración automáticos. Los grupos de Auto Scaling también permiten utilizar las características de Amazon EC2 Auto Scaling, como los reemplazos de comprobación de estado y las políticas de escalado. Tanto el mantenimiento del número de instancias en un grupo de Auto Scaling como el escalado automático son la funcionalidad principal del servicio Amazon EC2 Auto Scaling.

#### Uso de la selección de tipos de instancia basada en atributos y grupos de instancia mixtos

La flexibilidad de las instancias es una práctica recomendada importante de Spot. Puede utilizar la selección de tipo de instancia basada en atributos (ABIS) para seleccionar automáticamente varios tipos de instancia que se ajusten a sus requisitos. Un caso habitual de uso de grupos de Auto Scaling es con cargas de trabajo que requieren una combinación de capacidad spot y bajo demanda.

En este paso se crea un archivo json para crear grupos de Auto Scaling mediante la CLI de AWS. La configuración utiliza la plantilla de lanzamiento que creó en los pasos anteriores y ABIS para elegir cualquier tipo de instancia no GPU de generación actual con 2 vCPU y sin límite de memoria. OnDemandBaseCapacity permite establecer una capacidad inicial de 1 instancia bajo demanda. La capacidad restante es una mezcla de un 25% de instancias bajo demanda y un 75% de instancias spot definidas por el OnDemandPercentageAboveBaseCapacity.

In [5]:
# Specify the JSON file path
policy_file = "asg-policy.json"

# Read the JSON data from the file
with open(policy_file, 'r') as file:
    data = json.load(file)

data['LaunchTemplate']['LaunchTemplateSpecification']['LaunchTemplateId'] = launch_template_id

# Write the modified JSON data back to the file
with open(policy_file, 'w') as file:
    json.dump(data, file, indent=2)

En esta configuración se establece *SpotAllocationStrategy* como precio-capacidad-optimizado. La estrategia de asignación optimizada para precio y capacidad asigna instancias de los grupos de instancias de subasta que ofrecen precios bajos y alta disponibilidad de capacidad. Puede obtener más información sobre la estrategia de asignación optimizada para la capacidad de precio en la entrada del blog Introducción a la estrategia de asignación optimizada para la capacidad de precio para las instancias de subasta de EC2.

In [6]:
# Find the default VPC ID
vpcs = ec2.describe_vpcs(Filters=[{'Name': 'isDefault', 'Values': ['true']}])
if 'Vpcs' in vpcs:
    vpc_id = vpcs['Vpcs'][0]['VpcId']
else:
    print("No default VPC found.")
    vpc_id = None

if vpc_id:
    # Describe subnets for the VPC
    subnets_response = ec2.describe_subnets(Filters=[{'Name': 'vpc-id', 'Values': [vpc_id]}])
    
    # Extract the first two subnet IDs
    subnet_ids = [subnet['SubnetId'] for subnet in subnets_response['Subnets'][:2]]

    if len(subnet_ids) >= 2:
        subnet_1 = subnet_ids[0]
        subnet_2 = subnet_ids[1]
    else:
        print("Not enough subnets found in the VPC.")
        subnet_1 = None
        subnet_2 = None
else:
    subnet_1 = None
    subnet_2 = None

print("Default VPC ID:", vpc_id)
print("Subnet 1 ID:", subnet_1)
print("Subnet 2 ID:", subnet_2)

Default VPC ID: vpc-0336b7b9a020762c1
Subnet 1 ID: subnet-0606112a7ef40b226
Subnet 2 ID: subnet-0027e70008e24e870


Cree un grupo de Auto Scaling en 2 zonas de disponibilidad, tamaño mínimo 2, tamaño máximo 20 y capacidad deseada 10 unidades vCPU.

In [7]:
# Initialize the Auto Scaling client
autoscaling = boto3.client('autoscaling')

# Define the Auto Scaling group parameters
auto_scaling_group_name = 'EC2SpotWorkshopASG'
min_size = 2
max_size = 20
desired_capacity = 10
desired_capacity_type = 'vcpu'
subnet_ids = [subnet_1, subnet_2]
capacity_rebalance = True

# Load the mixed instances policy from a JSON file
with open(policy_file, 'r') as policy_file:
    mixed_instances_policy = json.load(policy_file)

# Create the Auto Scaling group
response = autoscaling.create_auto_scaling_group(
    AutoScalingGroupName=auto_scaling_group_name,
    MinSize=min_size,
    MaxSize=max_size,
    DesiredCapacity=desired_capacity,
    DesiredCapacityType=desired_capacity_type,
    VPCZoneIdentifier=','.join(subnet_ids),
    CapacityRebalance=capacity_rebalance,
    MixedInstancesPolicy=mixed_instances_policy
)
if response['ResponseMetadata']['HTTPStatusCode'] == 200:
    print("Auto Scaling group created successfully.")

Auto Scaling group created successfully.


**¿Cuántos grupos de instancias de spot tiene en cuenta el grupo de Autoescalado al aplicar la diversificación de spot?**

Un grupo de capacidad de subasta es un conjunto de instancias EC2 no utilizadas con el mismo tipo de instancia (por ejemplo, m5.large) y zona de disponibilidad. En el momento de la creación del taller, nuestro ejemplo coincidía con 35 tipos de instancia y 2 zonas de disponibilidad, lo que hace un total de (35*2)=70 pools de subasta. Aumentar el número de grupos de subasta es clave para adoptar las mejores prácticas de subasta.

**¿Cuántas instancias spot vs bajo demanda han sido solicitadas por el grupo Auto Scaling?**

La capacidad deseada es de 10 vCPUs, por lo que se aprovisionan 5 instancias que suman 10 vCPUs. De ellas, la primera instancia EC2 es On-Demand según lo solicitado por OnDemandBaseCapacity. El resto de las instancias, siguen una proporción de 25% On-Demand (1 instancia) y 75% Spot (3 instancias) según OnDemandPercentageAboveBaseCapacity.

Confirmar qué instancias se han creado en el grupo de Autoescalado:

In [ ]:
# Waite 15 seconds for the instances to be created
time.sleep(15)
response = autoscaling.describe_auto_scaling_groups(AutoScalingGroupNames=[auto_scaling_group_name])
if 'Instances' in response['AutoScalingGroups'][0]:
    for instance in response['AutoScalingGroups'][0]['Instances']:
        print(instance['InstanceId'])
        print(instance['InstanceType'])
        print("-------------------------------")
else:
    print("No instances created.")

**¿Cómo puede seleccionar manualmente tipos de instancia específicos en lugar de ABIS en su grupo de Autoescalado?**

Para crear un grupo de Autoescalado con tipos de instancia específicos/individuales, puede utilizar un archivo json que se indica a continuación. El ejemplo utiliza m5.large, c5.large, r5.large, m4.large, c4.large y r4.large.

In [ ]:
new_asg_policy = {
   "LaunchTemplate":{
      "LaunchTemplateSpecification":{
         "LaunchTemplateId":launch_template_id,
         "Version":"1"
      },
      "Overrides":[
         {
            "InstanceType":"m5.large"
         },
         {
            "InstanceType":"c5.large"
         },
         {
            "InstanceType":"r5.large"
         },
         {
            "InstanceType":"m4.large"
         },
         {
            "InstanceType":"c4.large"
         },
         {
            "InstanceType":"r4.large"
         }
      ]
   },
   "InstancesDistribution":{
      "OnDemandBaseCapacity":1,
      "OnDemandPercentageAboveBaseCapacity":25,
      "SpotAllocationStrategy":"price-capacity-optimized"
   }
}

with open("asg-policy_instances_types.json", 'w') as file:
    json.dump(new_asg_policy, file, indent=2)

**¿Cómo puede seleccionar una mezcla de tipos de instancia de diferentes tamaños en su grupo de Autoescalado?**

Para crear un grupo de Autoescalado con tipos de instancia específicos/individuales, puede utilizar el archivo json que se indica a continuación. Las instancias de ejemplo que tienen 2 vCPUs y 4 vCPUs son m5.large, c5.large, r5.large, m5.xlarge, c5.xlarge y r5.xlarge.

In [ ]:
new_asg_policy2 = {
   "LaunchTemplate":{
      "LaunchTemplateSpecification":{
         "LaunchTemplateId":launch_template_id,
         "Version":"1"
      },
      "Overrides":[{
         "InstanceRequirements": {
            "VCpuCount": {
               "Min": 2, 
               "Max": 4
            },
            "MemoryMiB": {
               "Min": 0
            },
            "CpuManufacturers": [
               "intel",
               "amd"
            ],
            "InstanceGenerations": [
               "current"
            ],
            "AcceleratorCount": {
               "Max": 0
            }
         }
      }]
   },
   "InstancesDistribution":{
      "OnDemandBaseCapacity":1,
      "OnDemandPercentageAboveBaseCapacity":25,
      "SpotAllocationStrategy":"capacity-optimized"
   }
}

with open("asg-policy_mixed_types.json", 'w') as file:
    json.dump(new_asg_policy2, file, indent=2)

**¿Cómo puede seleccionar la estrategia de asignación de Spot de capacidad optimizada en su grupo de Auto Scaling?**

La estrategia de asignación con capacidad optimizada asigna instancias de los grupos de instancias de subasta con la capacidad óptima para el número de instancias que se están lanzando, utilizando datos de capacidad en tiempo real y optimizando la selección de instancias de subasta utilizadas. La estrategia de asignación de Spot con capacidad optimizada funciona bien para cargas de trabajo en las que el coste de una interrupción de Spot es muy significativo. Puede consultar las ventajas de utilizar la optimización de la capacidad en la publicación del blog *[Capacity-Optimized Spot Instance allocation in action at Mobileye and Skyscanner](https://aws.amazon.com/blogs/aws/capacity-optimized-spot-instance-allocation-in-action-at-mobileye-and-skyscanner/)*.

In [ ]:
new_asg_policy3 = {
   "LaunchTemplate":{
      "LaunchTemplateSpecification":{
         "LaunchTemplateId":launch_template_id,
         "Version":"1"
      },
      "Overrides":[{
         "InstanceRequirements": {
            "VCpuCount": {
               "Min": 2, 
               "Max": 2
            },
            "MemoryMiB": {
               "Min": 0
            },
            "CpuManufacturers": [
               "intel",
               "amd"
            ],
            "InstanceGenerations": [
               "current"
            ],
            "AcceleratorCount": {
               "Max": 0
            }
         }
      }]
   },
   "InstancesDistribution":{
      "OnDemandBaseCapacity":1,
      "OnDemandPercentageAboveBaseCapacity":25,
      "SpotAllocationStrategy":"capacity-optimized"
   }
}

with open("asg-policy_capacity_optimized.json", 'w') as file:
   json.dump(new_asg_policy3, file, indent=2)

### Lecturas opcionales
Estas son algunas de las características y funcionalidades que hacen que los grupos de Amazon EC2 Auto Scaling sean la opción adecuada para la mayoría de las cargas de trabajo:

* Selección de tipo de instancia basada en atributos: Los grupos de Amazon EC2 Auto Scaling seleccionan una serie de familias y tamaños de instancias en función de un conjunto de atributos de instancia que describen sus requisitos informáticos. [Selección de tipo de instancia basada en atributos](https://docs.aws.amazon.com/autoscaling/ec2/userguide/create-asg-instance-type-requirements.html).

* Distribución de instancias y reequilibrio de zonas de disponibilidad: Los grupos de Amazon EC2 Auto Scaling intentan distribuir las instancias de manera uniforme para maximizar la alta disponibilidad de sus cargas de trabajo. [Distribución de instancias y reequilibrio de zonas de disponibilidad](https://docs.aws.amazon.com/autoscaling/ec2/userguide/auto-scaling-benefits.html#AutoScalingBehavior.Rebalancing).

* Escalado flexible: El grupo Auto Scaling dispone de un conjunto de APIs enriquecidas para administrar el escalado de tu carga de trabajo, permitiendo a las cargas de trabajo controlar sus necesidades de escalado sean cuales sean, desde Escalado manual, Escalado programado, Escalado dinámico mediante seguimiento de objetivos, Escalado por pasos y Escalado predictivo.

* Integración con Elastic Load Balancing: La integración con Elastic Load Balancing distribuye automáticamente el tráfico entrante de su aplicación entre todas las Instancias EC2 que esté ejecutando. Elastic Load Balancing y Amazon EC2 Auto Scaling.

* Actualización y sustitución de instancias en función de la vida útil máxima de la instancia: El grupo Auto Scaling reduce el trabajo pesado necesario al actualizar, por ejemplo, la AMI subyacente. La actualización de instancias permite a los usuarios actualizar gradualmente las instancias de un grupo de Auto Scaling. La sustitución de instancias también puede configurarse en función de la vida útil máxima de las instancias, lo que ayuda a los usuarios a aplicar las mejores prácticas de gobernanza.

* Protección de escalado: Permite proteger las instancias que aún están funcionando para que no sean seleccionadas para operaciones de escalado de finalización de instancias de Auto Scaling.

* Ganchos de ciclo de vida: Permitir que un grupo de Auto Scaling desencadene acciones para que los usuarios puedan administrar el ciclo de vida de las instancias del grupo de Auto Scaling. Ganchos del ciclo de vida de Amazon EC2 Auto Scaling.

* Reequilibrio de la capacidad: Amazon EC2 Auto Scaling conoce las notificaciones de recomendación de reequilibrio de instancias EC2. El servicio Amazon EC2 Spot emite estas notificaciones cuando las instancias spot corren un riesgo elevado de interrupción. Cuando el reequilibrio de capacidad está habilitado para un grupo de Auto Scaling, Amazon EC2 Auto Scaling intenta reemplazar de forma proactiva las instancias de subasta del grupo que han recibido una recomendación de reequilibrio, lo que ofrece la oportunidad de reequilibrar la carga de trabajo con nuevas instancias de subasta que no corren un riesgo elevado de interrupción.

* Pesos de instancia: Cuando configura un grupo de Auto Scaling para lanzar varios tipos de instancia, tiene la opción de definir el número de unidades de capacidad que cada instancia contribuye a la capacidad deseada del grupo, utilizando la ponderación de instancias. Esto le permite especificar el peso relativo de cada tipo de instancia de forma que se corresponda directamente con el rendimiento de su aplicación. Puede ponderar sus instancias para adaptarlas a las necesidades específicas de su aplicación, por ejemplo, por los núcleos (vCPUs) o por la memoria (GiBs). Pesos de grupo de EC2 Auto Scaling.

* Soporte para múltiples Launch Templates: El grupo Auto Scaling soporta múltiples Plantillas de Lanzamiento. Esto permite una flexibilidad extra en cómo se configura el grupo de Auto Scaling, por ejemplo soportando múltiples arquitecturas (i.e Graviton c6g e Intel c5) dentro de un mismo grupo de Auto Scaling. Documentación sobre múltiples plantillas de lanzamiento.

* Warm pools: Los Warm pools disminuyen la latencia de la obtención de capacidad en sus cargas de trabajo mediante la gestión de un pool de Instancias EC2 preinicializadas. Siempre que su aplicación necesite escalarse, el grupo de Auto Scaling puede recurrir al warm pool para satisfacer su nueva capacidad deseada. Warm pools para Amazon EC2 Auto Scaling.

### Sin capacidad Spot disponible

El error "no hay capacidad de subasta disponible" se produce cuando Amazon EC2 no tiene suficiente capacidad de subasta para satisfacer una solicitud de instancia de subasta o flota de subasta. La capacidad de subasta es la cantidad de capacidad informática de EC2 sobrante y no utilizada que está disponible para los clientes a un precio inferior al de las instancias bajo demanda. Para solucionar este error, realice una de las siguientes acciones:

* Mantenga la solicitud como está. La solicitud spot sigue realizando automáticamente la solicitud de lanzamiento hasta que haya capacidad disponible. Cuando la capacidad esté disponible, Amazon EC2 ejecutará la solicitud de subasta. Si se encuentra con el error "no hay capacidad de spot disponible" con frecuencia, considere utilizar la siguiente solución.

* Sea flexible con los tipos de instancia que solicita y las zonas de disponibilidad que despliega al configurar la carga de trabajo. Por ejemplo, en lugar de solicitar una instancia m5.large en us-east-1a, solicite una instancia m4.large, c5.large, r5.large o t3.xlarge en varias zonas de disponibilidad. Este tipo de solicitud aumenta las posibilidades de que Amazon Web Services (AWS) encuentre y asigne la cantidad necesaria de capacidad informática.

* Utilice la estrategia de asignación optimizada de precio y capacidad (práctica recomendada). Esta estrategia de asignación tiene en cuenta tanto el precio como la capacidad para seleccionar los grupos de instancias de spot. Los grupos de instancias de subasta seleccionados son los que tienen menos probabilidades de sufrir interrupciones y el precio más bajo posible. La estrategia optimizada de precio y capacidad mantiene una tasa de interrupción comparable a la estrategia de asignación optimizada de capacidad. Además, con esta estrategia, el precio total de sus instancias de spot suele ser inferior al de la estrategia de capacidad optimizada. Para obtener más información, consulte Estrategias de asignación para instancias de subasta.

* Utilice la estrategia de asignación de capacidad optimizada. Esta estrategia de asignación analiza los datos de capacidad en tiempo real para lanzar sus instancias de subasta en pools con la mayor capacidad disponible. La estrategia de asignación de capacidad optimizada reduce las posibilidades de recibir errores de "no hay capacidad de spot disponible".